In [1]:
# Disable TensorFlow debugging info and warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 2: Info and warnings not displayed 

In [2]:
import tensorflow as tf
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
import pathlib
import tqdm
import annotation, misc, hyspec_cnn, hyspec_io

In [3]:
# Disable GPUs
tf.config.set_visible_devices([], 'GPU')
visible_devices = tf.config.get_visible_devices()
for device in visible_devices:
    assert device.device_type != 'GPU'

In [4]:
# Parameters
train_frac = 0.7
train_classes_to_merge = {6:2,7:4,8:5}
val_classes_to_skip = [6,7,8]

In [5]:
# Paths
tiles_dir = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/4b_Rad_Georef_SGC_Tiles/TrainingAndValidationDatasets')
json_file = '/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/4b_Rad_Georef_SGC_Tiles/tile_classes.json'

In [6]:
# Read JSON with class names and labels
class_dict = annotation.read_class_dict(json_file)

In [7]:
# Load and merge datasets
dataset_dirs = [x for x in tiles_dir.glob('*/') if x.is_dir()]
merged_dataset = tf.data.experimental.load(str(dataset_dirs[0]))
for ds_dir in dataset_dirs[1:]:
    merged_dataset = merged_dataset.concatenate(tf.data.experimental.load(str(ds_dir)))

In [8]:
# Check how many tiles there are
n_tiles = merged_dataset.cardinality().numpy()
print(n_tiles)

459


In [9]:
# Shuffle dataset
shuffled_dataset = merged_dataset.shuffle(n_tiles)

In [10]:
# Split into training and validation datasets
n_train_tiles = int(train_frac*n_tiles)
train_ds = shuffled_dataset.take(n_train_tiles)
val_ds = shuffled_dataset.skip(n_train_tiles)

In [11]:
# test saving the shuffled dataset
# Reads from original files and saves to new location - nice!
# ReadCombineTilestest_save_path = '/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/X_SaveTensorflowDataset/test_shuffled_dataset'
# tf.data.experimental.save(shuffled_dataset,test_save_path)